In [2]:
import os

In [3]:
%pwd

'd:\\ML_Projects\\End-to-End ML Impleamentation\\End to end machine learning with mlflow\\reasearch'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\ML_Projects\\End-to-End ML Impleamentation\\End to end machine learning with mlflow'

In [ ]:
## Configuration entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [ ]:
## Configuration Manager 

from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml,create_directories

In [19]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
        

In [ ]:
## Data Ingestion
import os
import urllib.request as request
import zipfile
from src.mlproject.logging import logger
from src.mlproject.utils.common import get_size

In [21]:
class DataIngestion: 
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")

        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)


            


In [ ]:
## Data Ingestion Pipeline
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingetsion = DataIngestion(config=data_ingestion_config)
    data_ingetsion.download_file()
    data_ingetsion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-02 21:36:56,331: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-02 21:36:56,333: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-02 21:36:56,341: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-02 21:36:56,341: INFO: common: Created directory at: artifacts]
[2025-04-02 21:36:56,349: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-04-02 21:36:57,810: INFO: 3544682667: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C302:3FBC48:1F3A8B:484359:67ED60A0
Accept-Ranges: bytes
Date: Wed, 02

ModuleNotFoundError: No module named 'pandas'